In [3]:
import pdfplumber
import pandas as pd

In [ ]:
# Caminho para o arquivo dentro do projeto
caminho = '/home/danielbarbosa/seget/notebooks/arrecadacao/arquivos/arrecadacao_2025_02_25.pdf'

# Nome do arquivo de saída
saida_csv = '/home/danielbarbosa/seget/notebooks/arrecadacao/arquivos/consolidados/dados_arrecadacao_2025_02_25.csv'
saida_excel = '/home/danielbarbosa/seget/notebooks/arrecadacao/arquivos/consolidados/dados_arrecadacao_2025_02_25.xlsx'

# Definir colunas esperadas
colunas = ["Numcgm", "Nome", "Procedencia", "Tipo de debito", "Valor historico", "Valor corrigido", "Multa", "Juros", "Total"]

def extrair_dados_em_chunks(pdf_path, chunk_size=10):
    """
    Extrai dados de um PDF processando em chunks 
    de páginas para evitar sobrecarga de memória
    """

    with pdfplumber.open(caminho) as pdf:
        for i in range(0, len(pdf.pages), chunk_size):
            dados = []

            # Processar um conjunto de páginas de cada vez
            for page in pdf.pages[i:i+chunk_size]:
                table = page.extract_table()
                if table:
                    dados.extend(table)
            
            if dados:
                yield pd.DataFrame(dados, columns=colunas)

# Criar o arquivo CSV inicial com cabeçalho
pd.DataFrame(columns=colunas).to_csv(saida_csv, index=False)

# Criar arquivo Excel inicial e salvar cabeçalhos
with pd.ExcelWriter(saida_excel, engine="openpyxl", mode="w") as writer:
    pd.DataFrame(columns=colunas).to_excel(writer, sheet_name="dados_arrecadacao", index=False)

# Processar e salvar os chunks no CSV e Excel
with pd.ExcelWriter(saida_excel, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    for chunk_df in extrair_dados_em_chunks(caminho, chunk_size=200):
        chunk_df.to_csv(saida_csv, mode="a", header=False, index=False)
        chunk_df.to_excel(writer, sheet_name="dados_arrecadacao", index=False, header=False, startrow=writer.sheets["dados_arrecadacao"].max_row)

print(f'Processamento concluído! Dados salvos em {saida_csv}')